In [1]:
%cd ".."

import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

import tape_data
import munsell_data
import optimization as opt
from experiment import *
from polarization import phase_shift, transmission_mueller

/Users/kate/diy-hyperspectral


In [2]:
# Params 
n_coreset = 10     # Number of filters to choose
n_basis = 4        # Number of basis function to describe transmission spectra of filters
n_samples = 5000   # Number of randomly generated filters to choose from

# Sample theta randomly (n_meas = 0) or use preset list of thetas to generate samples
# thetas = np.linspace(0.0, 180.0, 10)
# n_meas = len(thetas)
n_meas = 0

wp_hd = Waveplate(tape_data.THICKNESS_HD, tape_data.BIREFRINGENCE_HD)
wp_ss = Waveplate(tape_data.THICKNESS_SS, tape_data.BIREFRINGENCE_SS)

In [3]:
wavelengths = munsell_data.MUNSELL_CHIPS_WAVELENGTHS

print("Creating dataset...", end = "")
dataset = opt.create_dataset(wavelengths, [wp_hd, wp_ss], n_samples = n_samples, thetas = None, fixed_waveplates = True)
print(" done.")

dataset_transmissions = np.array([config.transmission(wavelengths) for config in dataset.flatten()])

print("Choosing optimal filters...", end = "")
opt_filters, pca_basis = opt.choose_optimal_filters(dataset, wavelengths, n_coreset = n_coreset, n_meas = n_meas)
print(" done.")
opt_transmissions = np.array([config.transmission(wavelengths) for config in opt_filters])

# Project all filter transmissions down to basis function coefficients
filter_coefs, _, _, _ = np.linalg.lstsq(pca_basis.transpose(), dataset_transmissions.transpose(), rcond=None)

# Project optimal (coreset) filter transmissions down to basis function coefficients
coreset, _, _, _ = np.linalg.lstsq(pca_basis.transpose(), opt_transmissions.transpose(), rcond=None)

Creating dataset... done.
Choosing optimal filters... done.


In [4]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

plots = []
colors = ['Tealgrn', 'Purp', 'Oryel']

if n_basis == 2:
    scatter_all = go.Scatter(
        x = filter_coefs[0, :], y = filter_coefs[1, :],
        mode = 'markers',
        marker = dict(
            size = 10,
            color = '#bbbbbb',
            opacity = 0.4
        )
    )

    plots.append(scatter_all)

    for i in range(n_meas):
        start = i * n_coreset
        end = (i + 1) * n_coreset
        scatter_coreset = go.Scatter(
            x = coreset[0, start : end], y = coreset[1, start : end],
            mode = 'markers',
            marker = dict(
                size = 8,
                color = coreset[0, start : end],
                colorscale = colors[i],
                opacity = 1.
            )
        )

        plots.append(scatter_coreset)
        
    layout = go.Layout(
        margin = dict(l=0, r=0, b=0, t=0),
        xaxis_title = 'basis_0',
        yaxis_title = 'basis_1',
        showlegend = False
    )
else:
    scatter_all = go.Scatter3d(
        x = filter_coefs[0,:], y = filter_coefs[1,:], z = filter_coefs[2,:],
        mode = 'markers',
        name = 'all data points',
        marker = dict(
            size = 3,
            color = '#bbbbbb',
            opacity = 0.3
        )
    )

    plots.append(scatter_all)

    scatter_coreset = go.Scatter3d(
        x = coreset[0,:], y = coreset[1,:], z = coreset[2,:],
        mode = 'markers',
        marker = dict(
            size = 8,
            color = coreset[3, :],
            colorbar = dict(
                title = '4th dimension'
            ),
            colorscale = colors[0],
            opacity = 1.
        )
    )

    plots.append(scatter_coreset)

    # for i in range(n_meas):
    #     start = i * n_coreset
    #     end = (i + 1) * n_coreset

    #     scatter_coreset = go.Scatter3d(
    #         x = coreset[0, start : end], y = coreset[1, start : end], z = coreset[2, start : end],
    #         mode = 'markers',
    #         marker = dict(
    #             size = 8,
    #             color = coreset[3, start : end],
    #             colorbar = dict(
    #                 title = 'meas set ' + str(i + 1),
    #                 x = 1 + .1 * i
    #             ),
    #             colorscale = colors[i],
    #             opacity = 1.
    #         )
    #     )

    #     plots.append(scatter_coreset)

    layout = go.Layout(
        title = 'Optimal filter set',
        scene_xaxis_title = 'basis_0',
        scene_yaxis_title = 'basis_1',
        scene_zaxis_title = 'basis_2',
        showlegend = False
    )

fig = go.FigureWidget(data = plots, layout = layout)
fig

FigureWidget({
    'data': [{'marker': {'color': '#bbbbbb', 'opacity': 0.3, 'size': 3},
              'mode': …

In [6]:
print(opt_filters)

[Config(alphas = [1.29658409e+02 2.05874671e-02], theta = 75.06396084646332, waveplates = [Waveplate(th = 78740.0, bf = 0.00936), Waveplate(th = 66040.0, bf = 0.00872)])
 Config(alphas = [45.35125617 91.02176339], theta = 1.7220409881319187, waveplates = [Waveplate(th = 78740.0, bf = 0.00936), Waveplate(th = 66040.0, bf = 0.00872)])
 Config(alphas = [171.98756975 127.52766608], theta = 83.67230159190372, waveplates = [Waveplate(th = 78740.0, bf = 0.00936), Waveplate(th = 66040.0, bf = 0.00872)])
 Config(alphas = [92.73253881  3.42889193], theta = 2.9581876066994695, waveplates = [Waveplate(th = 78740.0, bf = 0.00936), Waveplate(th = 66040.0, bf = 0.00872)])
 Config(alphas = [104.76539574  52.80773013], theta = 179.54729869109931, waveplates = [Waveplate(th = 78740.0, bf = 0.00936), Waveplate(th = 66040.0, bf = 0.00872)])
 Config(alphas = [111.52632644 102.26162198], theta = 75.20884704066437, waveplates = [Waveplate(th = 78740.0, bf = 0.00936), Waveplate(th = 66040.0, bf = 0.00872)])
 